In [37]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats as stats

In [39]:
### CURRENT VERSION

# Load the CSV file from the data folder in the repo
df_timestamps = pd.read_csv('cleaned_data/data_web_exp.csv')

# Display the DataFrame
df_timestamps.shape

(318872, 6)

In [40]:
# Converting into datetime for later calculation

df_timestamps['date_time'] = pd.to_datetime(df_timestamps_clean['date_time'])

In [41]:
df_timestamps.head(2)

,client_id,visitor_id,visit_id,process_step,date_time,Variation
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07,Test
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51,Test


In [42]:
# Sorting values first by visit and then by time, to have timestamps of the same visit id group
# and the time in a single advancing direction

df_timestamps = df_timestamps.sort_values(by=['visit_id', 'date_time'])

In [77]:
df_timestamps.head(2)

,client_id,visitor_id,visit_id,process_step,date_time,Variation,previous_step,previous_date_time
106022,3561384,451664975_1722933822,100012776_37918976071_457913,confirm,2017-04-26 13:22:17,Test,NaN,NaT
106021,3561384,451664975_1722933822,100012776_37918976071_457913,confirm,2017-04-26 13:23:09,Test,confirm,2017-04-26 13:22:17


In [46]:
### Filtering DF for only Test group customers

df_timestamps_group_test = df_timestamps_clean[(df_timestamps_clean['Variation'] == "Test")]

In [47]:
## Filtering DF for only Test group customers

df_timestamps_group_control = df_timestamps_clean[(df_timestamps_clean['Variation'] == "Control")]

In [50]:
### Create a helper column, in order to access the previous step of the same visitor id

df_timestamps['previous_step'] = df_timestamps.groupby('visit_id')['process_step'].shift(1)
df_timestamps['previous_date_time'] = df_timestamps.groupby('visit_id')['date_time'].shift(1)

In [51]:
df_timestamps_group_test['previous_step'] = df_timestamps_clean_v_test.groupby('visit_id')['process_step'].shift(1)
df_timestamps_group_test['previous_date_time'] = df_timestamps_clean_v_test.groupby('visit_id')['date_time'].shift(1)

C:\Users\rafae\AppData\Local\Temp\ipykernel_58592\2903605953.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_timestamps_group_test['previous_step'] = df_timestamps_clean_v_test.groupby('visit_id')['process_step'].shift(1)
C:\Users\rafae\AppData\Local\Temp\ipykernel_58592\2903605953.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_timestamps_group_test['previous_date_time'] = df_timestamps_clean_v_test.groupby('visit_id')['date_time'].shift(1)


In [52]:
df_timestamps_group_control['previous_step'] = df_timestamps_clean_v_control.groupby('visit_id')['process_step'].shift(1)
df_timestamps_group_control['previous_date_time'] = df_timestamps_clean_v_control.groupby('visit_id')['date_time'].shift(1)

C:\Users\rafae\AppData\Local\Temp\ipykernel_58592\4037053500.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_timestamps_group_control['previous_step'] = df_timestamps_clean_v_control.groupby('visit_id')['process_step'].shift(1)
C:\Users\rafae\AppData\Local\Temp\ipykernel_58592\4037053500.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_timestamps_group_control['previous_date_time'] = df_timestamps_clean_v_control.groupby('visit_id')['date_time'].shift(1)


In [53]:
df_timestamps_group_control.head(5)

,client_id,visitor_id,visit_id,process_step,date_time,Variation,previous_step,previous_date_time
233507,105007,676020267_36602664238,100030127_47967100085_936361,start,2017-03-22 11:07:49,Control,NaN,NaT
26884,5623007,700426174_33289797318,100037962_47432393712_705583,start,2017-04-14 16:41:51,Control,NaN,NaT
26883,5623007,700426174_33289797318,100037962_47432393712_705583,start,2017-04-14 16:43:16,Control,start,2017-04-14 16:41:51
26882,5623007,700426174_33289797318,100037962_47432393712_705583,step_1,2017-04-14 16:43:55,Control,start,2017-04-14 16:43:16
26881,5623007,700426174_33289797318,100037962_47432393712_705583,start,2017-04-14 16:44:03,Control,step_1,2017-04-14 16:43:55


In [54]:
# This function calculates the time spent between steps
# and flags individual action as being an error (or not)

def get_time_spent(row):
    step_list = ["start", "step_1", "step_2", "step_3", "confirm"]
    
    # Get the current and previous steps
    current_step = row["process_step"]
    previous_step = row["previous_step"]

    # Ensure the current step is valid and find the expected previous step
    if current_step in step_list:
        current_pos_index = step_list.index(current_step)
        
        # If it's not the first step, find the expected previous step
        if current_pos_index > 0:
            expected_previous_step = step_list[current_pos_index - 1]
        else:
            expected_previous_step = None
    else:
        expected_previous_step = None

    # We consider it to not be an error until provided otherwise (in the next step)
    row["error_flag"] = False
    
    # Validate that the previous step matches the expected step
    if previous_step == expected_previous_step:
        # Calculate the time difference if the previous step is valid
        row["time_spent_seconds"] = (row["date_time"] - row["previous_date_time"]).total_seconds()

    ### Edge Case: A start follows after a confirm; not an Error case
    elif current_step == "start" and previous_step == "confirm":
        row["error_flag"] = False
        row["time_spent_seconds"] = None
    else:
        # If the previous step is not valid, set time_spent_seconds to NaN or some default value
        row["time_spent_seconds"] = None

        #Turns the flag into True (signaling that it is an error, unless the step is a start)
        #if current_step == "start":
        if not previous_step and current_step == "start":
            row["error_flag"] = np.nan
        else:
            row["error_flag"] = True
        
    return row

In [55]:
### Executing previous function through an apply, affecting the entirety of both DF (Test and Group)

df_timestamps_group_test = df_timestamps_group_test.apply(get_time_spent, axis=1)

df_timestamps_group_control = df_timestamps_group_control.apply(get_time_spent, axis=1)

In [56]:
df_timestamps_group_test.head(2)

,client_id,visitor_id,visit_id,process_step,date_time,Variation,previous_step,previous_date_time,error_flag,time_spent_seconds
106022,3561384,451664975_1722933822,100012776_37918976071_457913,confirm,2017-04-26 13:22:17,Test,NaN,NaT,True,NaN
106021,3561384,451664975_1722933822,100012776_37918976071_457913,confirm,2017-04-26 13:23:09,Test,confirm,2017-04-26 13:22:17,True,NaN


In [57]:
df_timestamps_group_control.head(2)

,client_id,visitor_id,visit_id,process_step,date_time,Variation,previous_step,previous_date_time,error_flag,time_spent_seconds
233507,105007,676020267_36602664238,100030127_47967100085_936361,start,2017-03-22 11:07:49,Control,NaN,NaT,True,NaN
26884,5623007,700426174_33289797318,100037962_47432393712_705583,start,2017-04-14 16:41:51,Control,NaN,NaT,True,NaN


In [58]:
# Absolute Amount of Errors in Test Group

df_timestamps_group_test['error_flag'].value_counts()

error_flag
False    102904
True      73661
Name: count, dtype: int64

In [59]:
### % Percentage of Errors in Test
df_timestamps_group_test['error_flag'].value_counts()[1] / df_timestamps_group_test['error_flag'].value_counts()[0] * 100

C:\Users\rafae\AppData\Local\Temp\ipykernel_58592\3839667693.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_timestamps_group_test['error_flag'].value_counts()[1] / df_timestamps_group_test['error_flag'].value_counts()[0] * 100


71.58225141879811

In [60]:
# Absolute Amount of Errors in Control Group

df_timestamps_group_control['error_flag'].value_counts()

error_flag
False    84428
True     57879
Name: count, dtype: int64

In [61]:
### % Percentage of Errors in Control
df_timestamps_group_control['error_flag'].value_counts()[1] / df_timestamps_clean_v_control['error_flag'].value_counts()[0] * 100

C:\Users\rafae\AppData\Local\Temp\ipykernel_58592\2148566792.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_timestamps_group_control['error_flag'].value_counts()[1] / df_timestamps_clean_v_control['error_flag'].value_counts()[0] * 100


68.55427109489742

In [62]:
# Average Time spent per Step, in seconds

mean_time_per_step_group_test = df_timestamps_group_test.groupby('process_step')['time_spent_seconds'].mean()

mean_time_per_step_group_test

process_step
confirm    105.093187
start             NaN
step_1      31.087069
step_2      37.144365
step_3      86.547344
Name: time_spent_seconds, dtype: float64

In [63]:
# Average Time spent per Step, in seconds

mean_time_per_step_group_control = df_timestamps_clean_v_control.groupby('process_step')['time_spent_seconds'].mean()

mean_time_per_step_group_control

process_step
confirm    127.461757
start             NaN
step_1      37.701655
step_2      33.864246
step_3      87.178790
Name: time_spent_seconds, dtype: float64

In [64]:
## Renaming indexes for reability

mean_time_per_step_group_test.index = ['D. step 3 to confirm', 'Z. before to start', 'A. start to step_1', 'B. step_1 to step_2', 'C. step_2 to step_3']

mean_time_per_step_group_control.index = ['D. step 3 to confirm', 'Z. before to start', 'A. start to step_1', 'B. step_1 to step_2', 'C. step_2 to step_3']

In [66]:
### Mean Time in the Test Group
mean_time_per_step_group_test

D. step 3 to confirm    105.093187
Z. before to start             NaN
A. start to step_1       31.087069
B. step_1 to step_2      37.144365
C. step_2 to step_3      86.547344
Name: time_spent_seconds, dtype: float64

In [67]:
### Mean of Mean of all Steps, Test Group

mean_time_per_step_group_test.mean()

64.96799139003879

In [68]:
### Mean Time in the Control Group

mean_time_per_step_group_control

D. step 3 to confirm    127.461757
Z. before to start             NaN
A. start to step_1       37.701655
B. step_1 to step_2      33.864246
C. step_2 to step_3      87.178790
Name: time_spent_seconds, dtype: float64

In [69]:
### Mean of Mean of all Steps, Control Group

mean_time_per_step_group_control.mean()

71.55161201514193

In [70]:
df_timestamps_clean_v_control.head(2)

,client_id,visitor_id,visit_id,process_step,date_time,Variation,previous_step,previous_date_time,error_flag,time_spent_seconds
233507,105007,676020267_36602664238,100030127_47967100085_936361,start,2017-03-22 11:07:49,Control,NaN,NaT,True,NaN
26884,5623007,700426174_33289797318,100037962_47432393712_705583,start,2017-04-14 16:41:51,Control,NaN,NaT,True,NaN


In [71]:
### Two Sample T-Test of independent Samples to test individual steps

# Hx0: new design does not impact time spent per step x
# Hx1: new design impacts time spent per step x

# Steps to evaluate, excluding start due to no step before it
steps = ["step_1", "step_2", "step_3", "confirm"]

for step in steps:
    # Filter data for each group and step
    test_group = df_timestamps_group_test[(df_timestamps_group_test['Variation'] == 'Test') & (df_timestamps_group_test['process_step'] == step)]
    control_group = df_timestamps_group_control[(df_timestamps_group_control['Variation'] == 'Control') & (df_timestamps_group_control['process_step'] == step)]

    # Drop null values
    test_group = test_group.dropna(subset=["time_spent_seconds"])
    control_group = control_group.dropna(subset=["time_spent_seconds"])   

    # Conduct an independent two-sample t-test for each step
    t_stat, p_value = stats.ttest_ind(test_group["time_spent_seconds"], control_group["time_spent_seconds"], equal_var=False)  # Set equal_var=True if variances are equal

    # Print results
    print(f"Step {step}: t-statistic = {t_stat:.3f}, p-value = {p_value:.3f}")
    
    # Interpretation
    if p_value < 0.05:  # Assuming a significance level of 0.05
        print(f"Step {step}: There is a significant difference between Test and Control.\n")
    else:
        print(f"Step {step}: No significant difference between Test and Control.\n")


Step step_1: t-statistic = -9.728, p-value = 0.000
Step step_1: There is a significant difference between Test and Control.

Step step_2: t-statistic = 6.504, p-value = 0.000
Step step_2: There is a significant difference between Test and Control.

Step step_3: t-statistic = -0.788, p-value = 0.431
Step step_3: No significant difference between Test and Control.

Step confirm: t-statistic = -12.889, p-value = 0.000
Step confirm: There is a significant difference between Test and Control.



In [72]:
### One-Way ANOVA to test and control group over all steps

# H0: new design does not impact time spent for the entiretiy of the process
# H1: new design impacts time spent for the entiretiy of the process

# Steps to evaluate
steps = ["step_1", "step_2", "step_3", "confirm"]

# Create empty lists to store time spent for test and control groups across all steps
test_times = []
control_times = []

for step in steps:
    # Filter data for each group and step
    test_group = df_timestamps_group_test[(df_timestamps_group_test['Variation'] == 'Test') & (df_timestamps_group_test['process_step'] == step)]
    control_group = df_timestamps_group_control[(df_timestamps_group_control['Variation'] == 'Control') & (df_timestamps_group_control['process_step'] == step)]

    # Drop null values
    test_group = test_group.dropna(subset=["time_spent_seconds"])
    control_group = control_group.dropna(subset=["time_spent_seconds"])

    # Append time spent for each step to the corresponding list
    test_times.extend(test_group["time_spent_seconds"].tolist())
    control_times.extend(control_group["time_spent_seconds"].tolist())

# Conduct a one-way ANOVA for the combined data across all steps
f_stat, p_value = stats.f_oneway(test_times, control_times)

# Print the results for the overall ANOVA across all steps
print(f"Overall ANOVA comparison across all steps: F-statistic = {f_stat:.3f}, p-value = {p_value}")

# Interpretation
if p_value < 0.05:  # Assuming a significance level of 0.05
    print("There is a significant difference between Test and Control across all steps.\n")
else:
    print("No significant difference between Test and Control across all steps.\n")


Overall ANOVA comparison across all steps: F-statistic = 189.395, p-value = 4.522805493525243e-43
There is a significant difference between Test and Control across all steps.



#### Conclusion

Why is the F-Statistic So High Despite Step 3’s Non-significance?
ANOVA focuses on the overall differences between the groups, rather than just individual steps. Even though Step 3 does not show a significant difference, the significant results at Steps 1, 2, and Confirm are substantial enough that they outweigh the lack of difference at Step 3, resulting in the large F-statistic.

To Re-Explain the F-Statistic of 189 in This New Context:
"The F-statistic of 189.395 reflects the fact that, overall, the Test and Control groups have significantly different time spent across most steps of the process. Specifically, large differences in time spent at Step 1, Step 2, and the Confirm step drive this strong statistical signal. Even though there is no significant difference at Step 3, the substantial differences at the other steps result in a large F-statistic. This indicates that, across the process as a whole, the Test and Control groups behave very differently, with the Test group often spending less time than the Control group."

In summary, the F-statistic gives us a broad, overall view of the differences between the two groups. It indicates that when we look at all the steps combined, the Test and Control groups are very different, largely driven by significant differences in the majority of the steps.